In [24]:
import pandas as pd
from os import listdir
from os.path import isfile, join

prompts_path = '../../data/prompt_outputs/'
raw_pred_fns = [int(f.replace('.txt', '')) for f in listdir(prompts_path) if isfile(join(prompts_path, f))]
sorted_fns = sorted(raw_pred_fns)

def read_prompt_outputs(file_name):
    numbers = []
    max_candlestick = 12
    
    try:
        with open(file_name, 'r') as file:
            for line in file:
                try:
                    number = float(line.strip())
                    numbers.append(number)
                except ValueError:
                    print(f"Skipping invalid line: {line.strip()}")

    except FileNotFoundError:
        print(f"File not found: {file_name}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return (max(numbers[:max_candlestick]), abs(min(numbers[:max_candlestick])))

# sorted_fns = sorted_fns[:10]
exp_dta = []
for ts in sorted_fns:
    file_name = f'../../data/prompt_outputs/{ts}.txt'
    max_, min_ = read_prompt_outputs(file_name)
    exp_dta.append({'ts': ts, 'max': max_, 'min': min_})

pd.DataFrame(exp_dta).to_csv('../../data/exp_dta.csv', index=False)